In [ ]:
!pip install textattack

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 51.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
import numpy as np
import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_recipes import TextFoolerJin2019
from textattack.datasets import Dataset
from textattack.augmentation import EasyDataAugmenter, EmbeddingAugmenter
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import matplotlib.pyplot as plt
from datasets import load_dataset

# Load the full dataset
dataset = load_dataset("rotten_tomatoes", split="train")

# Define the model
model_name = "textattack/distilbert-base-uncased-rotten-tomatoes"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Define the attack recipe
attack_recipe = TextFoolerJin2019.build(model_wrapper)

# Define the training set sizes (logarithmic scale)
training_set_sizes = [10**2, 10**3, 10**4]

# Define the augmentation methods
augmentation_methods = ["Baseline", "EDA", "Embedding"]

# Define the results dictionary
results = {size: {} for size in training_set_sizes}
confidence_intervals = {size: {} for size in training_set_sizes}

# Multiple runs for confidence intervals
num_runs = 5

# Loop through the training set sizes and augmentation methods
for training_set_size in training_set_sizes:
    for augmentation_method in augmentation_methods:
        # Store multiple run results
        method_results = []

        for _ in range(num_runs):
            # Randomly sample from the dataset
            np.random.seed()  # Different seed each time
            actual_size = min(training_set_size, len(dataset))
            subset_indices = np.random.choice(len(dataset), actual_size, replace=False)
            subset_dataset = dataset.select(subset_indices)

            # Prepare dataset based on augmentation method
            if augmentation_method == "Baseline":
                dataset_ = Dataset([(example['text'], example['label']) for example in subset_dataset])
            elif augmentation_method == "EDA":
                augmenter = EasyDataAugmenter(transformations_per_example=1)
                augmented_dataset = []
                for example in subset_dataset:
                    augmented_text = augmenter.augment(example['text'])[0]
                    augmented_dataset.append((augmented_text, example['label']))
                dataset_ = Dataset(augmented_dataset)
            elif augmentation_method == "Embedding":
                augmenter = EmbeddingAugmenter(transformations_per_example=1)
                augmented_dataset = []
                for example in subset_dataset:
                    augmented_text = augmenter.augment(example['text'])[0]
                    augmented_dataset.append((augmented_text, example['label']))
                dataset_ = Dataset(augmented_dataset)

            # Create an attacker object with the current dataset
            attacker = textattack.Attacker(attack_recipe, dataset_)

            # Perform the attack
            attack_results = attacker.attack_dataset()

            # Calculate success rate
            success_rate = sum(1 for r in attack_results if isinstance(r, textattack.attack_results.SuccessfulAttackResult)) / len(attack_results)
            method_results.append(success_rate)

        # Store results with confidence intervals
        results[training_set_size][augmentation_method] = np.mean(method_results)
        confidence_intervals[training_set_size][augmentation_method] = np.std(method_results)

# Prepare data for plotting
success_rates = {method: [results[size][method] for size in training_set_sizes] for method in augmentation_methods}
confidence_intervals_plot = {method: [confidence_intervals[size][method] for size in training_set_sizes] for method in augmentation_methods}

# Plot the results
plt.figure(figsize=(10, 6))
for method in augmentation_methods:
    plt.errorbar(training_set_sizes, success_rates[method],
                 yerr=confidence_intervals_plot[method],
                 fmt='-o', capsize=5, label=method)

plt.xscale('log')
plt.title('Success Rate of Adversarial Attacks by Training Set Size')
plt.xlabel('Training Set Size (log scale)')
plt.ylabel('Attack Success Rate')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set i

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

...

--------------------------------------------- Result 6 ---------------------------------------------
[[0 (54%)]] --> [[1 (50%)]]

an [[occasionally]] interesting but mostly repetitive look at a slice of counterculture that possible be best forgotten .

an [[sometimes]] interesting but mostly repetitive look at a slice of counterculture that possible be best forgotten .




[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:  70%|███████   | 7/10 [07:07<03:03, 61.04s/it]

--------------------------------------------- Result 7 ---------------------------------------------
[[1 (94%)]] --> [[0 (54%)]]

peppering this [[urban]] studies with references to norwegian folktales , villeneuve [[creates]] in maelstrom a [[world]] where the surreal is [[credible]] and the real turns magical .

peppering this [[stadt]] studies with references to norwegian folktales , villeneuve [[prescribes]] in maelstrom a [[worid]] where the surreal is [[conclusive]] and the real turns magical .




[Succeeded / Failed / Skipped / Total] 6 / 0 / 2 / 8:  80%|████████  | 8/10 [08:34<02:08, 64.27s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (92%)]] --> [[0 (51%)]]

fans of nijinsky will [[savor]] every minute of cox's [[works]] .

fans of nijinsky will [[delicacies]] every minute of cox's [[job]] .


